# Importar librerias

In [1]:
c = 20 # Valor para variables climaticas

In [38]:
import time
#importar datos y librerias

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA


from scipy import signal
from scipy.ndimage.interpolation import shift

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split
# from pmdarima.arima import auto_arima
# from statsmodels.tsa.arima_model import ARIMA
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
%matplotlib inline

from statsmodels.tsa.seasonal import seasonal_decompose
from xgboost import XGBRegressor
import cupy as cp

In [3]:
# Plotly
import chart_studio.plotly as ply
import cufflinks as cf
cf.go_offline()

# Funciones

**Mape**

In [4]:
# para calcular el mape
def Mape(y,y_pred):
    return np.mean(np.abs((y-y_pred)/y))*100

 **Kronecker**

In [5]:
# para hacer kronecker por fila sin GPU
def kronecker(data1:'Dataframe 1',data2:'Dataframe 2'):
    Combinacion = pd.DataFrame(); d1 = pd.DataFrame()
    for i in data2.columns:
        d1 = data1.multiply(data2[f"{i}"] , axis="index")
        d1.columns = [f'{i}_{j}' for j in data1.columns]
        Combinacion = pd.concat([Combinacion, d1], axis = 1)
    return Combinacion

# para hacer kronecker por fila con GPU
def corc(data1:'Dataframe 1',data2:'Dataframe 2'):
    nom = [f'{i}_{j}' for i in data2.columns for j in data1.columns] # nombres

    a = cp.array(data1).T
    b = cp.array(data2)
    cp.cuda.Stream.null.synchronize() #conversion a Cupy

    m, k = a.shape
    d = cp.zeros((k, 1))
    cp.cuda.Stream.null.synchronize() #vector de ceros para multiplicar

    for i in range(m):
        c = cp.multiply(a[[i]].T, b)
        d = cp.concatenate((d, c), 1)
        cp.cuda.Stream.null.synchronize() #kronecker

    d = cp.asnumpy(d)       #conversion a numpy
    d = np.delete(d, 0, 1)  #eliminar columna de ceros
    bob = pd.DataFrame(d, columns = nom) #pandas
    return bob

**Variables difusas**

In [6]:
# para crear variables difusas
def triangular():
    
    # Se crea x
    ma = pd.DataFrame();
    x = consumofeb.index.dayofyear.to_list()
    # Se crean variables a, b y c
    height = difs()
    for hei in height:
        ba = pd.DataFrame(x, columns=["x"])
        ba["a"] = height[hei][0]
        ba["b"] = height[hei][1]
        ba["c"] = height[hei][2]
        # Variable difusa
        ca = pd.DataFrame()
        ca["bo"] = (ba.x-ba.a)/(ba.b-ba.a)
        ca["ba"] = (ba.c-ba.x)/(ba.c-ba.b)
        ca = ca.min(axis = 1)
        ca[ca<0]=0
        ma = pd.concat([ma, ca], axis = 1) # Se concatenan

    ma.index = consumofeb.index
    ma.columns = height.keys()
    return ma

# Cargar datos

In [7]:
def cargar():
    # variables climaticas
    consumofeb=pd.read_excel(r'./Consumo_feb.xlsx').set_index("fecha").loc["2007-01-01":"2020-03-30"] #desde 2007 hasta 31 de marzo de 2020
    consumofeb = consumofeb.drop(["prom", 'Lluvia', 'Velocidad_viento'], axis = 1)
    #variables de consumo MWh
    consumo=pd.read_excel('./Data1.xlsx').set_index("fecha").loc["2007-01-01":"2020-03-30"] 
    return consumo, consumofeb

consumo, consumofeb = cargar()

In [8]:
if len(consumofeb.index) == len(consumo.index): print("ok") # confirmar que ambos documentos tengan la misma longitud

ok


In [9]:
# consumofeb.tail(20)

-  Para graficar

# Variables climaticas

In [10]:
fft_list = np.fft.fft(consumo.MWh)
for num_ in [100]:
    fft_list_m10 = np.copy(fft_list); fft_list_m10[num_:-num_]=0
    va = np.fft.ifft(fft_list_m10)
print(va.shape)

(4838,)


In [11]:
# vale = consumo.copy(); vale["d"] = np.abs(va); vale.iplot()

**Variables difusas** 

In [12]:
def difs():
    # Inicializar posicion de datos

    inv1 = [310.5-365.25, 8.6529, 138.941176470588]
    pr = [8.6529, 138.941176470588, 221.9375]
    vr = [138.941176470588, 221.9375, 310.5]
    ot = [221.9375, 310.5, 365.25+8.6529]
    ot2 = [221.9375-365.25, 310.5-365.25, 8.6529]
    inv2 = [310.5, 365.25+8.6529, 365.25+138.941176470588]

    height = {'Invierno1': inv1,'Primavera': pr,'verano': vr,'Otoño': ot,'Otoño2': ot2,'Invierno2': inv2,} 
    return height

In [13]:
# triangular().iplot()

In [14]:
def pba(j, n):
    # armonicos
    f, asd = signal.periodogram(consumofeb[j], 1)
#     plt.figure()
#     plt.plot(1/f, asd)
    picos = pd.DataFrame(asd, 1/(f), columns=["potencia"])
    picos = picos.sort_values(by="potencia",ascending=False).reset_index().head(12)
    picos.columns = ["periodo", "potencia"]
    #fourier
    sencos = pd.DataFrame(index = consumofeb.index)
    t = np.arange(1,len(consumofeb)+1)
    sencos["t"]=t
    for i  in  picos.periodo:
            sencos[f"{i:.2f}_sen"] = np.abs(np.sin(((2*np.pi)/(i))*t))
            sencos[f"{i:.2f}_cos"] = np.abs(np.cos(((2*np.pi)/(i))*t))
    sencos['ones']=1
    sencos['sen1']=np.abs(np.sin(((2*np.pi)/(365.25))*t))
    sencos['cos1']=np.abs(np.cos(((2*np.pi)/(365.25))*t))
    
    sencos=kronecker(sencos, triangular())
    #regresion
    X_train, X_test, y_train, y_test = train_test_split(sencos, consumofeb[j], test_size=n, shuffle = False)
    
    reg = LinearRegression(n_jobs=-1).fit(X_train, y_train)
    y_predict=reg.predict(X_test)
    
#     plt.figure()
#     plt.plot(y_test , label="real")
#     plt.plot(y_test.index, y_predict,label="predicción")

#     plt.figure()
#     plt.plot(y_train)
#     plt.plot(y_train.index, reg.predict(X_train))
#     print(Mape(y_train,reg.predict(X_train),1))
    
    nov=pd.DataFrame(np.concatenate([reg.predict(X_train),y_predict]),index=consumofeb.index,columns=[f"{j}"])

    return nov

In [15]:
# pba("pca1", 1)

In [16]:
def modelosclima(n):
    ca = pd.DataFrame(index = consumo.index)
    for i in ["mín", "max", "Nubosidad"]:
        ca = pd.concat([ca, pba(i, n)], axis = 1)
    return ca

In [17]:
# Se corre y guarda en una variable los datos de clima predichos
modclima = modelosclima(c)

# Variables festivos

In [18]:
def festivos():
    festivos=pd.read_excel('./Festivos.xlsx')
    festivos2=pd.read_excel('./Festivos2.xlsx')

    Dum = pd.DataFrame(index = consumo.index)

    for col in festivos.columns:
        Dum[f"{col}"] = consumo.index.isin(festivos[f"{col}"])
        Dum[col] = Dum[col].replace([False, True],[0, 1])
        clean = Dum[col].values
        Dum[col] = Dum[col]+shift(clean, 1, cval=0)*.4
        Dum[col] = Dum[col]+shift(clean, -1, cval=0)*.4
        Dum[col] = Dum[col]+shift(clean, 2, cval=0)*.1
        Dum[col] = Dum[col]+shift(clean, -2, cval=0)*.1
        Dum[col] = Dum[col]/2

    Dum["ones"] = 1
    tm = np.arange(1,len(consumo.index)+1)  
    Dum["t"] = tm/np.max(tm)
    
    for col in festivos2.columns:
        Dum[col] = consumo.index.isin(festivos2[col])
    Dum = Dum.replace([False, True],[0, 1])
    
    return Dum

In [19]:
# festivos()

# Todas las variables dependientes (dummies)

In [20]:
def dummies(dia, n):   
    # variables eureka y clima
    consumo2 = pd.DataFrame(index=consumo.index)
    consumo2["num"] = np.arange(1,len(consumo.index)+1)
    consumo2["day"] = consumo.index.weekday
    consumo2["eureka1"] = consumo2.day**3
    consumo2["eureka2"] = modclima["max"]*consumo2.num
    consumo2["month"] = consumo.index.month
    consumo2["year"] = consumo.index.year
    X = consumo2.join(modclima)
    # normalizacion eureka-clima
    X_norm = X/X.max()
    
    # variables gplearn
    X_norm["gplearn1"]=np.cos(X_norm.day)*X_norm.num
    X_norm["gplearn2"]=X_norm.num*X_norm["mín"]
    X_norm["gplearn3"]=3.319**X_norm["max"]*np.cos(X_norm.day)
    X_norm["gplearn8"]=3.319**X_norm["max"]*X_norm.num 
    X_norm["gplearn4"]=np.cos(X_norm.day)*np.sin(X_norm.day)
    X_norm["gplearn6"]=X_norm["max"]**2*X_norm["mín"]**2
    X_norm["gplearn7"]=1/np.cos(X_norm.day)
    X_norm["gplearn10"]=X_norm["max"]*X_norm["mín"]**3*np.cos(X_norm.month)**2
    X_norm = X_norm/X_norm.max()
    
    # variables dummies
    X_norm = pd.get_dummies(X_norm, columns=["month", "year"], prefix=["month", "year"], drop_first=True)
    X_norm = X_norm.drop(["day"], axis = 1)
    # variables festivos y difusa de clima
    X_norm = X_norm.join(festivos())
    X_norm = X_norm.join(triangular())
    
    # variables entre y fin
#     X_norm["fin"]=consumo.index.weekday
#     X_norm["entre"]=consumo.index.weekday
#     X_norm["fin"]=X_norm["fin"].replace([0,1,2,3,4,5,6],[.5,0,0,0,.5,1,1])
#     X_norm["entre"]=X_norm["entre"].replace([0,1,2,3,4,5,6],[.5,1,1,1,.5,0,0])
    return X_norm[X_norm.index.weekday == dia]

# Separar train-test

In [21]:
def separar(n, X, y):
    # separar X y Y
    X_train, X_test, y_train, y_test = train_test_split(X, y, \
                                                  test_size = n, shuffle = False)
    return (X_train, X_test), (y_train, y_test)

# Primer regresion

In [64]:
def zeta(dia, n):   
    # variables eureka y clima
    consumo2 = pd.DataFrame(index=consumo.index)
    consumo2["num"] = np.arange(1,len(consumo.index)+1)
    consumo2["day"] = consumo.index.weekday
    consumo2["eureka1"] = consumo2.day**3
    consumo2["eureka2"] = modclima["max"]*consumo2.num
    consumo2["month"] = consumo.index.month
    X = consumo2.join(modclima)
    # normalizacion eureka-clima
    X_norm = X/X.max()
    
    # variables gplearn
    X_norm["gplearn1"]=np.cos(X_norm.day)*X_norm.num
    X_norm["gplearn2"]=X_norm.num*X_norm["mín"]
    X_norm["gplearn3"]=3.319**X_norm["max"]*np.cos(X_norm.day)
    X_norm["gplearn8"]=3.319**X_norm["max"]*X_norm.num 
    X_norm["gplearn4"]=np.cos(X_norm.day)*np.sin(X_norm.day)
    X_norm["gplearn6"]=X_norm["max"]**2*X_norm["mín"]**2
    X_norm["gplearn7"]=1/np.cos(X_norm.day)
    X_norm["gplearn10"]=X_norm["max"]*X_norm["mín"]**3*np.cos(X_norm.month)**2
    # variables dummies
#     X_norm = pd.get_dummies(X_norm, columns=["month", "day"], prefix=["month", "day"], drop_first=True)

    # variables festivos y difusa de clima
#     X_norm = X_norm.join(festivos())
#     X_norm = X_norm.join(triangular())
    
    # variables entre y fin
#     X_norm["fin"]=consumo.index.weekday
#     X_norm["entre"]=consumo.index.weekday
#     X_norm["fin"]=X_norm["fin"].replace([0,1,2,3,4,5,6],[.5,0,0,0,.5,1,1])
#     X_norm["entre"]=X_norm["entre"].replace([0,1,2,3,4,5,6],[.5,1,1,1,.5,0,0])
    return X_norm[X_norm.index.weekday == dia]

In [69]:
def reg(dia, n, pre_forc): # n = 7 , pre_for = "general" o "forcasting"
    # cargar X y Y
    y = consumo[consumo.index.weekday == dia]; X = dummies(dia, n)
    X[["gplearn8", "eureka2"]]
    # Se obtienen los datos separados
    Xo, yo = separar(n, X, y)  
    # Decision "general" "forcasting"
    if pre_forc == "general": pre_forc = 0
    elif pre_forc == "forcasting": pre_forc = 1
    else: print(f"expected `general` or `forcasting`, got {pre_forc} instead")
    # regresion y predeccion    
    reg = LinearRegression(n_jobs =-1).fit(Xo[0], yo[0])
#     reg = XGBRegressor(max_depth = 5000, n_estimators = 1000, objective='reg:squarederror').fit(Xo[0],yo[0])
    y_predict=reg.predict(Xo[pre_forc])
    
    # dataframe real-predict- resta
    comp = pd.DataFrame(y_predict, columns = ["predict"],index = yo[pre_forc].index)
    comp["real"] = yo[pre_forc]; comp["resta"] = comp.real - comp.predict
    comp["error"] = np.abs(((comp.real - comp.predict))/comp.real)*100
    return comp

In [70]:
# def reg(dia, n, pre_forc): # n = 7 , pre_for = "general" o "forcasting"
#     # cargar X y Y
#     y = consumo[consumo.index.weekday == dia]; X = dummies(dia, n)
#     # Se obtienen los datos separados
#     Xo, yo = separar(n, X, y)  
#     # Decision "general" "forcasting"
#     if pre_forc == "general": pre_forc = 0
#     elif pre_forc == "forcasting": pre_forc = 1
#     else: print(f"expected `general` or `forcasting`, got {pre_forc} instead")
#     # regresion y predeccion    
#     reg = LinearRegression(n_jobs =-1).fit(Xo[0], yo[0])

#     xgbModel = XGBRegressor(max_depth = 250, n_estimators = 5, objective='reg:squarederror').fit(Xo[0],yo[0], eval_set = [(Xo[0], yo[0]), (Xo[pre_forc], yo[pre_forc])], verbose=False)
#     fig = plt.figure(figsize=(16,4))
#     plt.xticks(rotation='vertical')
#     plt.bar([i for i in range(len(xgbModel.feature_importances_))], xgbModel.feature_importances_.tolist(), tick_label=Xo[pre_forc].columns)
#     plt.title('Figure 6: Feature importance of the technical indicators.')
#     plt.show()
#     return pd.DataFrame(xgbModel.feature_importances_, index = Xo[pre_forc].columns, columns = ["bob"])

In [71]:
for i in range(7):
    bob = reg(i, 7, "general")
    print(bob.error.mean())

2.3149312908027126
2.1829433138010454
2.171529945998732
2.226304011689469
2.2826044496610964
2.229083842600399
2.2142569992859262


In [82]:
def aro(dia, n):
    X = zeta(dia, n)#; y_train = reg(dia, n, "general")[["real"]]; y_test = reg(dia, n, "forcasting")[["real"]]
    X_train, X_test, y_train, y_test = train_test_split(X, consumo[consumo.index.weekday == dia], test_size = n, shuffle = False)
    model = ARIMA(y_train.values, order=(7,1,0), exog = X_train.values)
    model_fit = model.fit(disp=0)
    output = model_fit.forecast(steps = n, exog = X_test.values); predo = output[0]
    y_test["predict"] = predo
    y_test["error"] = (np.abs(y_test.real - y_test.predict)/y_test.real)*100
    return y_test

In [83]:
aro(1, 7)

ValueError: x contains a constant. Adding a constant with trend='c' is not allowed.

In [50]:
for i in range(7):
    bob = aro(i, 7)
    print(bob.error.mean())

ValueError: x contains a constant. Adding a constant with trend='c' is not allowed.

- Para graficar

In [66]:
def visual1(dia, n):
    regio = reg(dia, n, "general")
    rego = regio[["predict", "real"]]
    
    rego.iplot(title = f"general: {Mape(*rego.values.T)}")
    regio.resta.iplot(title = "resta")
    
    regal = reg(dia, n, "forcasting")[["predict", "real"]]
    regal.iplot(title = f"forcasting: {Mape(*regal.values.T)}")

In [67]:
# visual1(1, 7)

# Periodos en residuales

In [309]:
def perio(dia, n):
    # Periodograma de resta real - pred
    reg1 = reg(dia, n, "general")
    
#     f, asd = signal.periodogram(reg1.resta, 1)
    
#     # graficar
#     plt.plot(1/f, asd); plt.xlabel('periodo'); plt.ylabel('PSD')


    result = seasonal_decompose(reg1.resta, model="additive").seasonal.reset_index(drop = True)
    f, asd = signal.periodogram(result)



    # DataFrame con columna de potencia y columna de 
    picos = pd.DataFrame(asd, columns=["potencia"])
    picos["periodo"] = 1/f
    picos = picos.sort_values(by="potencia",ascending=False).reset_index(drop = True).head(12)
    return picos[:4]

In [248]:
perio(1, 7)

,potencia,periodo
0,1.328078e+08,8.658228
1,1.274461e+08,6.514286
2,6.564899e+07,3.717391
3,5.426551e+07,4.717241
4,5.307003e+07,12.905660
5,4.267856e+07,7.434783
6,3.234190e+07,4.000000
7,2.609387e+07,10.363636
8,2.276397e+07,5.181818
9,2.234021e+07,13.153846


# Solo SenCos

In [310]:
def sencos(dia, n):
     # Se llama a X y a Y
    # Se crea dataframe de fourier
    sencos = pd.DataFrame()
    t = np.arange(1,len(consumo[consumo.index.weekday == dia])+1)
    sencos["t"]=t
    
    # Mando llamar la funcion de armonicos
    p = perio(dia, n)
    
    # fourier
    for i  in  p.periodo:
            sencos["{}_sen".format(i)] = np.abs(np.sin(((2*np.pi)/(i))*t))
            sencos["{}_cos".format(i)] = np.abs(np.cos(((2*np.pi)/(i))*t))
    sencos['ones']=1
    sencos['sen']=np.abs(np.sin(((2*np.pi)/14)*t))
    sencos['cos']=np.abs(np.cos(((2*np.pi)/14)*t))
    sencos['sen1']=np.abs(np.sin(((2*np.pi)/(365.25))*t))
    sencos['cos1']=np.abs(np.cos(((2*np.pi)/(365.25))*t))
    sencos['sen2']=np.abs(np.sin(((2*np.pi)/(28))*t))
    sencos['cos2']=np.abs(np.cos(((2*np.pi)/(28))*t))
    sencos.index =consumo[consumo.index.weekday == dia].index
    return sencos

In [311]:
def reg2(dia, n, pre_forc): # n = 7 , pre_for = "general" o "forcasting"
    # Se llama a X y a Y
    X = dummies(dia, n); T = sencos(dia, n); y = consumo[consumo.index.weekday == dia]
    C = pd.concat([X, T], 1)
    # split
    Xo, yo = separar(n, C, y) 
    
    # Decision "general" "forcasting"
    if pre_forc == "general": pre_forc = 0
    elif pre_forc == "forcasting": pre_forc = 1
    else: print(f"expected `general` or `forcasting`, got {pre_forc} instead")
        
    # regresion y predeccion    
    reg = LinearRegression(n_jobs =-1).fit(Xo[0], yo[0])
#     reg = Lasso(normalize = True, max_iter = 10000).fit(Xo[0], yo[0])


    y_predict=reg.predict(Xo[pre_forc])
    
    # dataframe real-predict- resta
    comp = pd.DataFrame(y_predict, columns = ["predict"],index = yo[pre_forc].index)
    comp["real"] = yo[pre_forc]; comp["error"] = np.abs(((comp.real - comp.predict))/comp.real)*100
    comp["resta"] = comp.real - comp.predict
    return comp

In [312]:
for i in range(7):
    print(reg2(i, 7, "general").error.mean())

2.2252753354704
2.1064743898207383
2.0996441804977444
2.1631452453632423
2.203056295857972
2.169491236226695
2.154674078905796


In [244]:
bob = reg2(0, 7, "general")
bob[["predict", "real"]].iplot(title = bob.error.mean())

# Kronecker Sencos-dummies

In [292]:
def kron(dia, n):
    # Se llama a X y a Y
    X = dummies(dia, n) 
    # Se crea dataframe de fourier
    sencos = pd.DataFrame()
    t = np.arange(1,len(consumo[consumo.index.weekday == dia])+1)
    sencos["t"]=t
    
    # Mando llamar la funcion de armonicos
    p = perio(dia, n)[2:3]
    
    # fourier
    for i  in  p.periodo:
            sencos["{}_sen".format(i)] = np.abs(np.sin(((2*np.pi)/(i))*t))
            sencos["{}_cos".format(i)] = np.abs(np.cos(((2*np.pi)/(i))*t))
    sencos['ones']=1
    sencos['sen']=np.abs(np.sin(((2*np.pi)/14)*t))
    sencos['cos']=np.abs(np.cos(((2*np.pi)/14)*t))
    sencos['sen1']=np.abs(np.sin(((2*np.pi)/(365.25))*t))
    sencos['cos1']=np.abs(np.cos(((2*np.pi)/(365.25))*t))
    sencos['sen2']=np.abs(np.sin(((2*np.pi)/(28))*t))
    sencos['cos2']=np.abs(np.cos(((2*np.pi)/(28))*t))
    sencos.index =consumo[consumo.index.weekday == dia].index
    
    # Kronecker
    combination=kronecker(X, sencos)
    return combination

# Segunda regresion

In [293]:
def reg2(dia, n, pre_forc): # n = 7 , pre_for = "general" o "forcasting"
    # Se llama a X y a Y
    X = kron(dia, n); y = consumo[consumo.index.weekday == dia]
    
    # split
    Xo, yo = separar(n, X, y) 
    
    # Decision "general" "forcasting"
    if pre_forc == "general": pre_forc = 0
    elif pre_forc == "forcasting": pre_forc = 1
    else: print(f"expected `general` or `forcasting`, got {pre_forc} instead")
        
    # regresion y predeccion    
    reg = LinearRegression(n_jobs =-1).fit(Xo[0], yo[0])
#     reg = Lasso(normalize = True, max_iter = 10000).fit(Xo[0], yo[0])


    y_predict=reg.predict(Xo[pre_forc])
    
    # dataframe real-predict- resta
    comp = pd.DataFrame(y_predict, columns = ["predict"],index = yo[pre_forc].index)
    comp["real"] = yo[pre_forc]; comp["resta"] = comp.real - comp.predict
    return comp

In [299]:
bay = reg2(2, 1, "forcasting")
Mape(bay.real, bay.predict)

47.01718788677745

In [300]:
bay[["real", "predict"]].iplot()

- para graficar

In [273]:
def visual2(dia, n):
    regio = reg2(dia, n, "general")
    rego = regio[["predict", "real"]]
    
    rego.iplot(title = f"general: {Mape(*rego.values.T)}")
    regio.resta.iplot(title = "resta")
    
    regal = reg2(dia, n, "forcasting")[["predict", "real"]]
    regal.iplot(title = f"forcasting: {Mape(*regal.values.T)}")

In [274]:
# visual2(1, 10)

# Tercera regresion

In [275]:
def rego(n):
    X = kron(n)
    X_train, X_test, y_train, y_test = train_test_split(X, consumo, test_size = n, shuffle = False)
    
    # regresion con lasso # 150, 5
    lasso_reg = XGBRegressor(n_jobs= -1, n_estimators=70, max_depth=3, tree_method='gpu_hist', nthread = -1, objective = "reg:squarederror").fit(X_train,y_train)
    y_pred_lass = lasso_reg.predict(X_test)

    # predict, real, resta
    comp = pd.DataFrame(y_pred_lass, columns = ["predict"],index = y_test.index)
    comp["real"] = y_test; comp["resta"] = comp.real - comp.predict
    return comp

In [276]:
beg = time.time()
ba = rego(20)
end = time.time()
print(f" timepo de corrida: {(end - beg)/60} min")

TypeError: kron() missing 1 required positional argument: 'n'

In [114]:
# ai = Mape(*ba[["predict", "real"]].values.T)
# ba[["predict", "real"]].plot(title = f"Mape: {ai:.3f}")

- para graficar

# Separado

**Para obtener multiples dataframes**

In [132]:
def banana(dia, n):
    X = kron(dia, n); y = consumo[consumo.index.weekday == dia]; cas ={}
    for i in range(1, 4):
        j = i*n
        print(j)

        X_train = X.head(len(y)-j); X_test = X.tail(j).head(n)
        y_train = y.head(len(y)-j); y_test = y.tail(j).head(n)

        XGB_reg = XGBRegressor(n_jobs= -1, n_estimators=70, max_depth=3, nthread = -1, objective = "reg:squarederror").fit(X_train,y_train)
        y_pred_xgb = XGB_reg.predict(X_test)

        comp = pd.DataFrame(y_pred_xgb, columns = ["predict"],index = y_test.index)
        comp["real"] = y_test; comp["resta"] = comp.real - comp.predict

        cas[j] = comp
    return cas

res = banana(1, 10)

10
20
30


In [133]:
for i in [10, 20, 30]:
    print(Mape(res[i].real, res[i].predict))

2.697683842313862
4.063274291343018
2.569889225787797


In [131]:
for i in [10, 20, 30]:
    print(Mape(res[i].real, res[i].predict))

3.206874291243618
3.601121181018755
2.9797198342656688


# PSO

In [34]:
from pyswarm import pso

In [57]:
def PSO(teef, ne):
    n = 7
    X = kron(n); y = consumo; mape =[]
    for i in range(1, 2):
        j = i*n
        print(i)

        X_train = X.head(len(y)-j); X_test = X.tail(j).head(n)
        y_train = y.head(len(y)-j); y_test = y.tail(j).head(n)

        XGB_reg = XGBRegressor(n_jobs= -1,n_estimators=int(ne), max_depth=int(teef), nthread = -1, reg_alpha = 1,\
                               objective = "reg:squarederror").fit(X_train,y_train)
        
        y_pred_xgb = XGB_reg.predict(X_test) # tree_method=’gpu_hist’

        mape.append(Mape(y_test.values, y_pred_xgb))
        print(sum(mape))

    return sum(mape)

# bon, boy = PSO(10, 70)

In [58]:
plt.plot(bon)
plt.plot(boy)

NameError: name 'bon' is not defined

In [59]:
# Aumenta de forma lineal con el teef: 100 10s - 1000 26s
# Es constante con n_estimadores : 100 2.24s - 1000 2.14s

In [60]:
def correr(hi):
    pro, num= hi
#     for deep, number, leaf
    a = time.time()
    mape = PSO(pro, num)
    b = time.time()
    c = b - a
    return mape*10 + c

In [61]:
correr([4, 5])

1
59.68863740319199


599.8316953534073

In [64]:

beg = time.time()
lb= [int(1), int(1)]; ub = [int(100), int(100)]
xopt, fopt = pso(correr, lb, ub, swarmsize=15,  maxiter=5, minstep=10) #, omega = 100
end = time.time()
print(end - beg)

1
6.717519524463472
1
6.7726597057280875
1
6.722148802055007
1
10.915625978528972
1
66.28215911817318
1
43.339165716706255
1
6.726059645246024
1
6.716868751325639
1
7.969722672897231
1
18.036250068506
1
25.237312676408685
1
6.740237313373106
1
6.7202791040529615
1
6.774516037062995
1
6.740094566814658
1
90.95613218852691
1
90.2332170627671
1
90.2332170627671
1
90.2332170627671
1
6.740158378020437
1
6.740158378020437
1


KeyboardInterrupt: 

In [63]:
xopt, fopt

(array([ 1., 10.]), 401.0319543492169)